<a href="https://colab.research.google.com/github/linlcc/NLP/blob/master/2_1_LanguageModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LanguageModeling

In this task, we will train *character-level* language models. 
You will train unigram, bigram, and trigram character level models on a collection of books from Project Gutenberg. You will then use these trained English language models to distinguish English documents from Brazilian Portuguese documents in the test set.

In [0]:
import math
import pandas as pd
import httpimport
from sklearn.feature_extraction.text import CountVectorizer

with httpimport.remote_repo(['lm_helper'], 'https://raw.githubusercontent.com/jasoriya/CS6120-PS2-support/master/utils/'):
  from lm_helper import get_train_data, get_test_data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package mac_morpho to /root/nltk_data...
[nltk_data]   Unzipping corpora/mac_morpho.zip.


This code loads the training and test data. Each dataset is a list of books. Each book contains a list of sentences, and each sentence contains a list of words. For building a character language model, you should join the words of a sentence together with a space character.

In [0]:
# get the train and test data
train = get_train_data()
test, test_files = get_test_data()

In [0]:
# prepare all the contents formated properly for the later use 

test_sec=[]
for item in test:
    test1=[]    
    for i in item:
        test1.append(' '.join(word for word in i))        
    test_sec.append(test1)
    
con=[]
corpus=[]
for i in train:
    for item in i:
        con.append(' '.join(word for word in item))
corpus.append(' '.join(con))

from sklearn.model_selection import train_test_split
training, held_out = train_test_split(con, test_size=0.2) ##training set (80%) and a held-out (20%)
training80=[]
training80.append(' '.join(training))

## 1.1
Collect statistics on the unigram, bigram, and trigram character counts.


In [0]:
count_vect = CountVectorizer(analyzer='char', ngram_range=(1, 1))
texts = count_vect.fit_transform(corpus) # creat sparse matrix of contect list
vocab=count_vect.get_feature_names() # list of the vocabularies

unigram=texts.toarray()
unigram_counts = dict(zip(vocab, unigram[0]))
# print(unigram_counts)
def keyfunction(k):
    return unigram_counts[k]
print('Top 10 character counts in unigram: ')
for key in sorted(unigram_counts, key=keyfunction, reverse=True)[:10]:
    print ("%s: %i" % (key, unigram_counts[key]))

Top 10 character counts in unigram: 
 : 2621784
e: 1119617
t: 827161
a: 731203
o: 678136
h: 650743
n: 615091
i: 577691
s: 556863
r: 502402


In [0]:
count_vect = CountVectorizer(analyzer='char', ngram_range=(2, 2))
texts = count_vect.fit_transform(corpus) # creat sparse matrix of contect list
vocab=count_vect.get_feature_names() # list of the vocabularies

bigram=texts.toarray()
bigram_counts = dict(zip(vocab, bigram[0]))
# print(bigram_counts)
def keyfunction(k):
    return bigram_counts[k]
print('Top 10 character counts in bigram: ')
for key in sorted(bigram_counts, key=keyfunction, reverse=True)[:10]:
    print ("%s: %i" % (key, bigram_counts[key]))

Top 10 character counts in bigram: 
e : 428723
 t: 354340
th: 330682
he: 291389
d : 268321
 a: 245575
t : 217685
s : 215867
 ,: 192106
, : 186347


In [0]:
count_vect = CountVectorizer(analyzer='char', ngram_range=(3, 3))
texts = count_vect.fit_transform(corpus) # creat sparse matrix of contect list
vocab=count_vect.get_feature_names() # list of the vocabularies

trigram=texts.toarray()
trigram_counts = dict(zip(vocab, trigram[0]))
# print(trigram_counts)
def keyfunction(k):
    return trigram_counts[k]
print('Top 10 character counts in trigram: ')
for key in sorted(trigram_counts, key=keyfunction, reverse=True)[:10]:
    print ("%s: %i" % (key, trigram_counts[key]))

Top 10 character counts in trigram: 
 th: 263231
the: 205546
 , : 186091
he : 168863
nd : 115035
 an: 110442
and: 108955
 of: 75678
 . : 73780
of : 72522


## 1.2
Calculate the perplexity for each document in the test set using linear interpolation smoothing method. For determining λs for linear interpolation, you can divide the training data into a new training set (80%) and a held-out set (20%), then using grid search method:
Choose ~10 values of λ to test using grid search on held-out data.

Some documents in the test set are in Brazilian Portuguese. Identify them as follows: 
  - Sort by perplexity and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names (from `test_files`) and perplexities of the documents above the threshold

    ```
        file name, score
        file name, score
        . . .
        file name, score
    ```

  - Copy this list of filenames and manually annotate them as being correctly or incorrectly labeled as Portuguese.




In [0]:
def ngram(x,y,z): #x=ngram, y=trainfile, z=lambda
    count_vect = CountVectorizer(analyzer='char', ngram_range=(x, x),token_pattern=r'\b[^\d\W]+\b')
    texts = count_vect.fit_transform(y) # creat sparse matrix of contect list
    vocab=count_vect.get_feature_names() # list of the vocabularies

    trigram=texts.toarray()
    trigram_counts = dict(zip(vocab, trigram[0]))

    trigram_counts = {k: (v+z) / (total+len(vocab)*z) for total in (sum(trigram_counts.values()),) for k, v in trigram_counts.items()}
    return trigram_counts

In [0]:
unigram_counts=ngram(1,training80,0)
bigram_counts=ngram(2,training80,0)
trigram_counts=ngram(3,training80,0)

In [0]:
def getvalue(x,y):
    a=x.get(y)
    if a is not None:
        return a
    else:
        return 10e-10

In [0]:
def perplexity_try(content,l1,l2,l3):
    total=0
    count=0    
    
    for j in content:
        count=count + len(j)

        for i in range(len(j)):
            if i==0:
                a=getvalue(unigram_counts,(j[i].lower()))
                b=10e-10
                c=10e-10
            elif i==1:    
                a=getvalue(unigram_counts,(j[i].lower()))
                b=getvalue(bigram_counts,(j[i-1].lower()+j[i].lower()))
                c=10e-10
            else:    
                a=getvalue(unigram_counts,(j[i].lower()))     
                b=getvalue(bigram_counts,(j[i-1].lower()+j[i].lower()))
                c=getvalue(trigram_counts,(j[i-2].lower()+j[i-1].lower()+j[i].lower()))
            
            k=l1*a+l2*b+l3*c
            total=total+math.log(k,2)     
            
    perplexity=pow(2,((-1/count)*total))

    return perplexity

In [0]:
# 10 values of λ
lset=[[0.8,0.1,0.1],[0.7,0.2,0.1],[0.6,0.2,0.2],[0.5,0.3,0.2],[0.4,0.5,0.1],
      [0.3,0.5,0.2],[0.2,0.6,0.2],[0.2,0.7,0.1],[0.2,0.3,0.5],[0.1,0.3,0.6]]

In [0]:
# grid search for the best combination of λ

for item in lset:
    print(item[0],item[1],item[2],perplexity_try(held_out,item[0],item[1],item[2]))

0.8 0.1 0.1 23.071218962945615
0.7 0.2 0.1 25.638068577134934
0.6 0.2 0.2 29.463385661521066
0.5 0.3 0.2 33.855647559976994
0.4 0.5 0.1 38.92781795857396
0.3 0.5 0.2 48.77551189653424
0.2 0.6 0.2 63.351709305282434
0.2 0.7 0.1 61.261704630461
0.2 0.3 0.5 71.07245938514383
0.1 0.3 0.6 114.8418006291888


In [0]:
# calulate perplexities of the documents and show files name which is above the threshold=35

i=0
thresould=35
for item in range(len(test_sec)):
    k=perplexity_try(test_sec[item],0.8,0.1,0.1)
    if k>=thresould:
        i=i+1
        print( test_files[item],"%.2f" % k)
print('Total above the thresould: '+ str(i))

br94ma01.txt 38.80
ag94ja11.txt 40.14
ag94ju07.txt 42.60
ag94fe1.txt 39.83
br94ju01.txt 39.38
br94ag01.txt 40.97
br94fe1.txt 41.50
ag94de06.txt 41.43
br94de01.txt 39.29
ag94ma03.txt 42.53
br94ja04.txt 41.00
ag94jl12.txt 41.86
br94ab02.txt 38.70
br94jl01.txt 39.75
ag94no01.txt 42.05
ag94ag02.txt 41.98
ag94mr1.txt 41.31
ag94ou04.txt 40.99
ag94ab12.txt 42.85
ag94se06.txt 43.39
Total above the thresould: 20


In [0]:
# manually output all the file names contain "txt" which is Portuguese

matching=[s for s in test_files if "txt" in s]
print('Total Portuguese files in test set: '+str(len(matching)))
print(matching)

Total Portuguese files in test set: 20
['br94ma01.txt', 'ag94ja11.txt', 'ag94ju07.txt', 'ag94fe1.txt', 'br94ju01.txt', 'br94ag01.txt', 'br94fe1.txt', 'ag94de06.txt', 'br94de01.txt', 'ag94ma03.txt', 'br94ja04.txt', 'ag94jl12.txt', 'br94ab02.txt', 'br94jl01.txt', 'ag94no01.txt', 'ag94ag02.txt', 'ag94mr1.txt', 'ag94ou04.txt', 'ag94ab12.txt', 'ag94se06.txt']


Compare the total number of Portuguese files is exactually the same as the files above threshold=35.

## 1.3
Build a trigram language model with add-λ smoothing (use λ = 0.1).

Sort the test documents by perplexity and perform a check for Brazilian Portuguese documents as above:

  - Observe the perplexity scores and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names and perplexities of the documents above the threshold

  ```
      file name, score
      file name, score
      . . .
      file name, score
  ```

  - Copy this list of filenames and manually annotate them for correctness.

In [0]:
# trigram model with λ = 0.1
trigram_counts=ngram(3,training80,0.1)

In [0]:
def perplexity_tri(content):
    total=0
    count=0    
    
    for j in content:
        count=count + len(j)
        for i in range(2,len(j)):  
            c=getvalue(trigram_counts,(j[i-2].lower()+j[i-1].lower()+j[i].lower()))          
            total=total+math.log(c,2)     
            
    perplexity=pow(2,((-1/count)*total))
    return perplexity

In [0]:
# calulate perplexities of the documents and show files name which is above the threshold=10000
i=0
thresould=10000
for item in range(len(test_sec)):
    k=perplexity_tri(test_sec[item])
    if k>=thresould:
        i=i+1
        print( test_files[item],"%.2f" % k)
print('Total above the thresould: '+ str(i))

br94ma01.txt 15089.75
ag94ja11.txt 15284.46
ag94ju07.txt 16987.75
ag94fe1.txt 14691.65
br94ju01.txt 14197.61
br94ag01.txt 15982.04
br94fe1.txt 15823.80
ag94de06.txt 16451.39
br94de01.txt 13676.63
ag94ma03.txt 17549.97
br94ja04.txt 15493.30
ag94jl12.txt 16248.47
br94ab02.txt 14081.14
br94jl01.txt 14286.58
ag94no01.txt 16207.51
ag94ag02.txt 16008.43
ag94mr1.txt 16239.16
ag94ou04.txt 15375.28
ag94ab12.txt 16071.63
ag94se06.txt 17499.64
Total above the thresould: 20


In [0]:
# manually output all the file names contain "txt" which is Portuguese

print('Total Portuguese files in test set: '+str(len(matching)))
print(matching)

Total Portuguese files in test set: 20
['br94ma01.txt', 'ag94ja11.txt', 'ag94ju07.txt', 'ag94fe1.txt', 'br94ju01.txt', 'br94ag01.txt', 'br94fe1.txt', 'ag94de06.txt', 'br94de01.txt', 'ag94ma03.txt', 'br94ja04.txt', 'ag94jl12.txt', 'br94ab02.txt', 'br94jl01.txt', 'ag94no01.txt', 'ag94ag02.txt', 'ag94mr1.txt', 'ag94ou04.txt', 'ag94ab12.txt', 'ag94se06.txt']


Compare the total number of Portuguese files is exactually the same as the files above threshold=35.

## 1.4
Based on your observation from above questions, compare linear interpolation and add-λ smoothing by listing out their pros and cons.

Based on your observation from above questions, compare linear interpolation and add-λ smoothing by listing out their pros and cons.

For the linear interpolation method, the overall perplexities are all small(under 50); For add-λ smoothing, the overall perplexities are way higher than the linear interpolation method (over 1000). Although both of the methods can correctly distinguish English documents and Brazilian Portuguese documents, but for lower perplexity in the linear interpolation method indicates the probability distribution is better at predicting the test set. However, since linear interpolation method need to consider unigram, bigram and also trigram, which means it needs to spend more time(at least 3 times) to build and train model than only trigram(add-λ smoothing) method. Therefore, both of them are workable for this test set, but if the documents are more complicated, we need to use linear interpolation method, otherwise we can only use trigram to predict.